### Imports

In [2]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
# from resnet.plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "OfficeHome"
MODEL_NAME = "WinKawaks/vit-small-patch16-224"

Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [3]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [4]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-20 11:21:02,249 | INFO | Initialized experiment directories for OfficeHome
2025-10-20 11:21:02,250 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\OfficeHome\logs
2025-10-20 11:21:02,251 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\OfficeHome\checkpoints
2025-10-20 11:21:02,251 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\OfficeHome\plots


### Setup

In [5]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]


{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [6]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-20 11:21:07,366 | INFO | === LODO: Leaving out domain 'Art' ===



=== LODO: Leaving out domain 'Art' ===


Evaluating: 100%|██████████| 19/19 [00:13<00:00,  1.37it/s]
2025-10-20 11:22:51,949 | INFO | [Art] Epoch 1/5 | Train - Loss: 1.5776, Cls: 1.5700, GRQO: 0.0075, Acc: 0.7127 | Val - Loss: 0.9159, Cls: 0.9143, GRQO: 0.0016, Acc: 0.7849
2025-10-20 11:22:52,054 | INFO | [Art] New best val acc: 0.7849


[Art] Epoch 1/5 | Train - Loss: 1.5776, Cls: 1.5700, GRQO: 0.0075, Acc: 0.7127 | Val - Loss: 0.9159, Cls: 0.9143, GRQO: 0.0016, Acc: 0.7849
[Art] New best val acc: 0.7849


Evaluating: 100%|██████████| 19/19 [00:29<00:00,  1.57s/it]
2025-10-20 11:25:26,138 | INFO | [Art] Epoch 2/5 | Train - Loss: 0.2916, Cls: 0.2879, GRQO: 0.0037, Acc: 0.9271 | Val - Loss: 0.9312, Cls: 0.9303, GRQO: 0.0009, Acc: 0.7660


...

[Real World] Epoch 4/5 | Train - Loss: 0.0934, Cls: 0.0907, GRQO: 0.0027, Acc: 0.9788 | Val - Loss: 0.4448, Cls: 0.4444, GRQO: 0.0004, Acc: 0.8820


2025-10-20 12:12:12,515 | INFO | [Real World] New best val acc: 0.8820


[Real World] New best val acc: 0.8820


Evaluating: 100%|██████████| 35/35 [01:55<00:00,  3.29s/it]
2025-10-20 12:16:00,439 | INFO | [Real World] Epoch 5/5 | Train - Loss: 0.0649, Cls: 0.0625, GRQO: 0.0024, Acc: 0.9842 | Val - Loss: 0.4783, Cls: 0.4780, GRQO: 0.0003, Acc: 0.8809
2025-10-20 12:16:00,439 | INFO | [Real World] Best Acc: 0.8820
2025-10-20 12:16:00,439 | INFO | ------------------------------------------------------------
2025-10-20 12:16:00,455 | INFO | LODO finished | Mean Acc: 0.8022 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\OfficeHome\logs\lodo_summary_20251020_121600.json


[Real World] Epoch 5/5 | Train - Loss: 0.0649, Cls: 0.0625, GRQO: 0.0024, Acc: 0.9842 | Val - Loss: 0.4783, Cls: 0.4780, GRQO: 0.0003, Acc: 0.8809
[Real World] Best Acc: 0.8820
------------------------------------------------------------
LODO finished | Mean Acc: 0.8022
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\OfficeHome\logs\lodo_summary_20251020_121600.json


### Baseline

In [7]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)


2025-10-20 12:16:00,478 | INFO | Initializing ViT baseline: WinKawaks/vit-small-patch16-224


Initializing ViT baseline: WinKawaks/vit-small-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([65]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([65, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-20 12:16:02,353 | INFO | === Baseline LODO: Leaving out domain 'Art' ===



=== Baseline LODO: Leaving out domain 'Art' ===


2025-10-20 12:19:38,010 | INFO | [Art] Epoch 1/5 | Train - Loss: 1.1126, Acc: 0.7525 | Val Acc: 0.7750


...

[Real World] Epoch 3/5 | Train - Loss: 0.1017, Acc: 0.9753 | Val Acc: 0.8768


2025-10-20 13:03:12,592 | INFO | [Real World] Epoch 4/5 | Train - Loss: 0.0576, Acc: 0.9862 | Val Acc: 0.8816


[Real World] Epoch 4/5 | Train - Loss: 0.0576, Acc: 0.9862 | Val Acc: 0.8816


2025-10-20 13:06:18,068 | INFO | [Real World] Epoch 5/5 | Train - Loss: 0.0437, Acc: 0.9881 | Val Acc: 0.8772
2025-10-20 13:06:18,068 | INFO | [Real World] Best Val Acc: 0.8816
2025-10-20 13:06:18,068 | INFO | ------------------------------------------------------------
2025-10-20 13:06:18,068 | INFO | Baseline LODO (WinKawaks/vit-small-patch16-224) finished | Mean Acc: 0.7993


[Real World] Epoch 5/5 | Train - Loss: 0.0437, Acc: 0.9881 | Val Acc: 0.8772
[Real World] Best Val Acc: 0.8816
------------------------------------------------------------
Baseline LODO (WinKawaks/vit-small-patch16-224) finished | Mean Acc: 0.7993
